In [ ]:
import numpy as np
from scipy import signal
import soundfile as sf
import sounddevice as sd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import queue
import threading
import time
import os
from datetime import datetime
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from ttkbootstrap.dialogs import Messagebox
from tkinterdnd2 import DND_FILES, TkinterDnD
from scipy.signal import butter, lfilter
class AudioFile:
    def __init__(self):
        self.audio_data = None
        self.sample_rate = None
        self.current_position = 0
        self.duration = 0
        self.is_playing = False
        self.filename = None
        
    def load_file(self, filename):
        try:
            audio_data, sample_rate = sf.read(filename)
            if len(audio_data.shape) > 1:
                audio_data = np.mean(audio_data, axis=1)
            
            self.audio_data = audio_data
            self.sample_rate = sample_rate
            self.current_position = 0
            self.duration = len(audio_data) / sample_rate
            self.filename = filename
            return True
        except Exception as e:
            print(f"Error loading audio file: {e}")
            return False
            
    def get_next_chunk(self, chunk_size):
        if self.current_position >= len(self.audio_data):
            return None
            
        end_pos = min(self.current_position + chunk_size, len(self.audio_data))
        chunk = self.audio_data[self.current_position:end_pos]
        self.current_position = end_pos
        
        if len(chunk) < chunk_size:
            chunk = np.pad(chunk, (0, chunk_size - len(chunk)))
            
        return chunk
        
    def seek(self, position):
        self.current_position = int(position * self.sample_rate)

class AudioPlayer:
    def __init__(self, sample_rate, frame_size):
        self.sample_rate = sample_rate
        self.frame_size = frame_size
        self.stream = None
        self.audio_file = AudioFile()
        
    def play(self, callback):
        if self.stream is None or not self.stream.active:
            self.stream = sd.OutputStream(
                channels=1,
                samplerate=self.sample_rate,
                blocksize=self.frame_size,
                callback=callback
            )
            self.stream.start()
            
    def stop(self):
        if self.stream is not None and self.stream.active:
            self.stream.stop()
            self.stream.close()
            self.stream = None
            
    def load_file(self, filename):
        return self.audio_file.load_file(filename)

class AudioVisualizer:
    def __init__(self, frame, sample_rate):
        self.sample_rate = sample_rate

        # Reduced figure size for better compactness
        self.fig = Figure(figsize=(8, 4))

        self.ax_wave = self.fig.add_subplot(211)
        self.ax_wave.set_title('Waveform')
        self.ax_wave.set_ylim(-1, 1)
        self.ax_wave.set_xlim(0, 1024)
        self.wave_line, = self.ax_wave.plot([], [], 'b-', lw=1)

        self.ax_spectrum = self.fig.add_subplot(212)
        self.ax_spectrum.set_title('Frequency Spectrum')
        self.ax_spectrum.set_ylim(-60, 20)
        self.ax_spectrum.set_xlim(20, 20000)
        self.ax_spectrum.set_xscale('log')
        self.spectrum_line, = self.ax_spectrum.plot([], [], 'g-', lw=1)

        self.canvas = FigureCanvasTkAgg(self.fig, master=frame)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        # Tight layout for reduced padding
        self.fig.tight_layout()
        
    def update(self, audio_data):
        self.wave_line.set_data(np.arange(len(audio_data)), audio_data)
        
        spectrum = np.fft.rfft(audio_data)
        freq = np.fft.rfftfreq(len(audio_data), 1/self.sample_rate)
        spectrum_db = 20 * np.log10(np.abs(spectrum) + 1e-10)
        self.spectrum_line.set_data(freq, spectrum_db)
        
        self.canvas.draw()

class ThreeBandEqualizer:
    def __init__(self):
        self.sample_rate = 44100
        self.frame_size = 1024
        self.audio_queue = queue.Queue(maxsize=10)
        
        # Define frequency bands
        self.nyquist = self.sample_rate / 2
        self.low_cutoff = [20 / self.nyquist, 200 / self.nyquist]
        self.mid_cutoff = [200 / self.nyquist, 2000 / self.nyquist]
        self.high_cutoff = [2000 / self.nyquist, 20000 / self.nyquist]
        
        # Initialize gains (in linear scale)
        self.gains = np.ones(3)
        
        # Number of taps for FIR filters
        self.num_taps = 101
        
        self.filters = self._design_filters()
        self.player = AudioPlayer(self.sample_rate, self.frame_size)
        
    def _design_filters(self):
        """Design FIR filters for each band using the improved method"""
        filters = []
        
        # Low-band filter (band-pass)
        filters.append(signal.firwin(self.num_taps, self.low_cutoff, 
                                   pass_zero=False, window='hamming'))
        
        # Mid-band filter (band-pass)
        filters.append(signal.firwin(self.num_taps, self.mid_cutoff, 
                                   pass_zero=False, window='hamming'))
        
        # High-band filter (band-pass)
        filters.append(signal.firwin(self.num_taps, self.high_cutoff, 
                                   pass_zero=False, window='hamming'))
        
        return filters
    
    def plot_frequency_responses(self):
        """Plot frequency response of all filters in a new popup window with zoom and pan capabilities."""
        # Create "Frequency response" folder if it doesn't exist
        folder_name = "Frequency response"
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        # Generate a unique filename with a timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        image_filename = f"frequency_response_{timestamp}.png"
        image_path = os.path.join(folder_name, image_filename)

        # Create a new top-level window for the plot
        popup = tk.Toplevel()  # Create a new top-level window
        popup.title("Frequency Response of All Bands")
        
        fig, ax = plt.subplots(figsize=(8, 4))
        colors = ['b', 'g', 'r']
        labels = ['Low', 'Mid', 'High']
        
        for i, (filt, color, label) in enumerate(zip(self.filters, colors, labels)):
            w, h = signal.freqz(filt, worN=8000)
            freq = (w / np.pi) * self.nyquist
            response = 20 * np.log10(np.abs(h))
            ax.semilogx(freq, response, color, label=f'{label} Band', alpha=0.7)
        
        ax.set_title('Frequency Response of All Bands')
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('Magnitude (dB)')
        ax.set_xlim(20, self.nyquist)
        ax.set_ylim(-60, 5)
        ax.grid(True)
        ax.legend()
        fig.tight_layout()

        # Save the plot as an image in the "Frequency response" folder with the unique timestamped filename
        fig.savefig(image_path)
        print(f"Frequency response saved at: {image_path}")
        
        # Display the plot in the Tkinter popup window
        canvas = FigureCanvasTkAgg(fig, master=popup)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

        # Set initial x and y limits and state for panning
        self.xlim = [20, self.nyquist]
        self.ylim = [-60, 5]
        self.pan_start = None  # Track start position for panning

        def zoom(event):
            """Zoom in/out on scroll."""
            base_scale = 1.1  # Zoom factor

            # Check scroll direction and adjust scale factor
            if event.delta > 0:  # Scroll up
                scale_factor = 1 / base_scale
            else:  # Scroll down
                scale_factor = base_scale

            # Update x-axis limits for zooming
            x_center = (self.xlim[0] + self.xlim[1]) / 2
            x_range = (self.xlim[1] - self.xlim[0]) * scale_factor
            self.xlim = [max(20, x_center - x_range / 2), min(self.nyquist, x_center + x_range / 2)]
            
            # Update y-axis limits for zooming
            y_center = (self.ylim[0] + self.ylim[1]) / 2
            y_range = (self.ylim[1] - self.ylim[0]) * scale_factor
            self.ylim = [y_center - y_range / 2, y_center + y_range / 2]

            # Apply updated limits and redraw the canvas
            ax.set_xlim(self.xlim)
            ax.set_ylim(self.ylim)
            canvas.draw()

        def start_pan(event):
            """Record the start position for panning."""
            self.pan_start = (event.x, event.y)

        def pan(event):
            """Pan the view on drag."""
            if self.pan_start is None:
                return

            # Calculate distance dragged
            dx = event.x - self.pan_start[0]
            dy = event.y - self.pan_start[1]

            # Convert pixels to data units
            x_range = self.xlim[1] - self.xlim[0]
            y_range = self.ylim[1] - self.ylim[0]
            
            # Adjust x and y limits based on drag distance and redraw
            width = canvas.get_tk_widget().winfo_width()
            height = canvas.get_tk_widget().winfo_height()
            self.xlim = [self.xlim[0] - dx * x_range / width,
                        self.xlim[1] - dx * x_range / width]
            self.ylim = [self.ylim[0] + dy * y_range / height,
                        self.ylim[1] + dy * y_range / height]

            ax.set_xlim(self.xlim)
            ax.set_ylim(self.ylim)
            canvas.draw()

            # Update start position for next pan event
            self.pan_start = (event.x, event.y)
        def apply_filters(self, audio_data):
            """Apply the three-band equalizer filters to the audio data"""
            filtered_audio = np.zeros_like(audio_data)
            for i in range(3):
                filtered_audio += signal.lfilter(self.filters[i], 1, audio_data) * self.gains[i]
            return filtered_audio
        def end_pan(event):
            """Reset the pan start position."""
            self.pan_start = None

        # Bind zoom and pan functions to the canvas
        canvas.get_tk_widget().bind("<MouseWheel>", zoom)         # Zoom with scroll
        canvas.get_tk_widget().bind("<ButtonPress-1>", start_pan) # Start panning on left click
        canvas.get_tk_widget().bind("<B1-Motion>", pan)           # Pan on drag
        canvas.get_tk_widget().bind("<ButtonRelease-1>", end_pan) # End panning on release

    def process_audio(self, audio_data):
        """Process audio through the three-band equalizer"""
        output = np.zeros_like(audio_data)
        
        for i in range(3):
            # Apply band-specific filter and gain
            filtered = signal.lfilter(self.filters[i], [1.0], audio_data)
            output += filtered * self.gains[i]
        
        # Normalize output to prevent clipping
        max_val = np.max(np.abs(output))
        if max_val > 1.0:
            output = output / max_val
            
        return output
        
    def process_full_audio(self):
        """Process the entire audio file with current equalizer settings"""
        if self.player.audio_file.audio_data is None:
            return None
            
        return self.process_audio(self.player.audio_file.audio_data)
        
    def audio_callback(self, outdata, frames, time, status):
        if status:
            print(status)
            
        chunk = self.player.audio_file.get_next_chunk(frames)
        
        if chunk is None:
            self.player.stop()
            self.player.audio_file.current_position = 0
            raise sd.CallbackStop()
            
        processed = self.process_audio(chunk)
        outdata[:] = processed.reshape(-1, 1)
        
        try:
            self.audio_queue.put_nowait(processed)
        except queue.Full:
            pass
            
    def set_gain(self, band_idx, gain_db):
        """Set gain for a specific band (in dB)"""
        self.gains[band_idx] = 10 ** (gain_db / 20)
        
    def save_processed_audio(self, output_filename):
        """Save the processed audio to a new file"""
        if self.player.audio_file.audio_data is None:
            return False
            
        try:
            processed_audio = self.process_full_audio()
            sf.write(output_filename, processed_audio, self.sample_rate)
            return True
        except Exception as e:
            print(f"Error saving audio file: {e}")
            return False
class RealTimeAudioApp:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Real-Time Audio Filter")

        self.equalizer = ThreeBandEqualizer()
        self.visualizer = AudioVisualizer(self.root, self.equalizer.sample_rate)

        self.start_button = ttk.Button(self.root, text="Start", command=self.start_recording)
        self.start_button.pack(pady=10)

        self.stop_button = ttk.Button(self.root, text="Stop", command=self.stop_recording)
        self.stop_button.pack(pady=10)

        self.is_recording = False

    def start_recording(self):
        self.is_recording = True
        self.stream = sd.InputStream(callback=self.audio_callback, channels=1, samplerate=self.equalizer.sample_rate)
        self.stream.start()

    def audio_callback(self, indata, frames, time, status):
        if status:
            print(status)
        if self.is_recording:
            audio_data = indata[:, 0]
            if hasattr(self.equalizer, 'apply_filters'):
                filtered_data = self.equalizer.apply_filters(audio_data)
                self.visualizer.update(filtered_data)
        else:
            print("Error: apply_filters method not found in equalizer.")
    def apply_filters(self, audio_data):
            """Apply the three-band equalizer filters to the audio data"""
            filtered_audio = np.zeros_like(audio_data)
            for i in range(3):
                filtered_audio += signal.lfilter(self.filters[i], 1, audio_data) * self.gains[i]
            return filtered_audio
    def stop_recording(self):
        self.is_recording = False
        self.stream.stop()

    def run(self):
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.root.mainloop()

    def on_closing(self):
        self.stop_recording()
        self.root.destroy()
class ThreeBandEqualizerGUI:
    def __init__(self, equalizer):
        # Initialize TkinterDnD root window
        # Initialize ttkbootstrap root window with a chosen theme
        self.root = TkinterDnD.Tk()
        self.root.title("Three-Band Equalizer")

        self.style = ttk.Style()
        self.style.theme_use("cosmo")  # Set a theme for ttkbootstrap
        
        self.equalizer = equalizer
        self.audio_duration = 0
        self.running = False

        main_frame = ttk.Frame(self.root)
        main_frame.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)

        # Create a drop zone frame with custom style
        style = ttk.Style()
        style.configure('Dropzone.TFrame', borderwidth=2, relief='solid')

        
        self.drop_frame = ttk.Frame(main_frame, style='Dropzone.TFrame')
        self.drop_frame.pack(side=tk.TOP, fill=tk.X, pady=5)

        # Drop zone label with theme-aware color
        self.drop_label = ttk.Label(
            self.drop_frame, 
            text="Drag and drop audio files here or click 'Open File'",
            padding=10,
            bootstyle="info"  # Adds the theme-aware color
        )
        self.drop_label.pack(pady=5)

        # Additional UI setup remains the same, but adjust styling:
        file_frame = ttk.Frame(main_frame)
        file_frame.pack(side=TOP, fill=X, pady=5)

        # Configure drop zone for the frame only (not the entire window)
        self.drop_frame.drop_target_register(DND_FILES)
        self.drop_frame.dnd_bind('<<Drop>>', self.handle_drop)
        self.drop_frame.dnd_bind('<<DragEnter>>', self.on_drag_enter)
        self.drop_frame.dnd_bind('<<DragLeave>>', self.on_drag_leave)

        # File controls frame
        file_frame = ttk.Frame(main_frame)
        file_frame.pack(side=tk.TOP, fill=tk.X, pady=5)

        ttk.Button(file_frame, text="Open File", command=self.open_file, bootstyle="primary").pack(side=LEFT, padx=5)
        ttk.Button(file_frame, text="Save Processed", command=self.save_file, bootstyle="secondary").pack(side=LEFT, padx=5)
        ttk.Button(file_frame, text="View Frequency Response", 
                  command=self.equalizer.plot_frequency_responses, bootstyle="success").pack(side=LEFT, padx=5)


        self.file_label = ttk.Label(file_frame, text="No file loaded", bootstyle="secondary")
        self.file_label.pack(side=tk.LEFT, padx=5)

        # Visualizer frame
        viz_frame = ttk.Frame(main_frame)
        viz_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        self.visualizer = AudioVisualizer(viz_frame, equalizer.sample_rate)

        # Progress bar frame
        progress_frame = ttk.Frame(main_frame)
        progress_frame.pack(side=tk.TOP, fill=tk.X, pady=10)

        self.progress = ttk.Scale(
            progress_frame, from_=0, to=100, orient=tk.HORIZONTAL,
            command=self.seek_audio
        )
        self.progress.pack(fill=tk.X, expand=True, side=tk.LEFT, padx=10)
        
        self.current_time_label = ttk.Label(progress_frame, text="00:00")
        self.current_time_label.pack(side=tk.LEFT)
        self.duration_label = ttk.Label(progress_frame, text="00:00")
        self.duration_label.pack(side=tk.RIGHT)

        # Control frame
        control_frame = ttk.Frame(main_frame)
        control_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=10)

        # Sliders with `info` style for consistent color
        slider_frame = ttk.Frame(control_frame)
        slider_frame.pack(side=tk.TOP, fill=tk.X)

        # Create sliders
        self.sliders = []
        labels = ["Bass", "Mid", "Treble"]
        for i, label in enumerate(labels):
            frame = ttk.Frame(slider_frame)
            frame.pack(side=tk.LEFT, padx=10)

            slider = ttk.Scale(
                frame, from_=12, to=-12, length=150, 
                orient=tk.VERTICAL,
                bootstyle="info",
                command=lambda x, i=i: self.update_gain(i, x)
            )
            slider.set(0)
            slider.pack()

            ttk.Label(frame, text=label, bootstyle="info").pack()
            self.sliders.append(slider)

        # Playback controls
        button_frame = ttk.Frame(control_frame)
        button_frame.pack(side=tk.BOTTOM, pady=15)

        ttk.Button(button_frame, text="Play", command=self.play_audio, bootstyle="success").pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Stop", command=self.stop_audio, bootstyle="danger").pack(side=tk.LEFT, padx=5)

        # Start threads
        self.running = True
        self.update_thread = threading.Thread(target=self.update_visualization)
        self.update_thread.daemon = True
        self.update_thread.start()

        self.progress_thread = threading.Thread(target=self.update_progress_bar)
        self.progress_thread.daemon = True
        self.progress_thread.start()

    def on_drag_enter(self, event):
        """Change appearance when file is dragged over"""
        self.drop_label.configure(text="Release to load audio file")
        style = ttk.Style()
        style.configure('Dropzone.TFrame', background='lightblue')
        self.drop_frame.configure(style='Dropzone.TFrame')

    def on_drag_leave(self, event):
        """Restore appearance when file is dragged away"""
        self.drop_label.configure(text="Drag and drop audio files here or click 'Open File'")
        style = ttk.Style()
        style.configure('Dropzone.TFrame', background='white')
        self.drop_frame.configure(style='Dropzone.TFrame')

    def handle_drop(self, event):
        """Handle dropped files"""
        file_path = event.data
        
        # Remove curly braces if present (Windows can add these)
        file_path = file_path.strip('{}')
        
        # Reset drop zone appearance
        self.on_drag_leave(event)
        
        # Check if it's an audio file
        if file_path.lower().endswith(('.wav', '.mp3', '.ogg')):
            if self.equalizer.player.load_file(file_path):
                self.file_label.config(text=os.path.basename(file_path))
                self.audio_duration = self.equalizer.player.audio_file.duration
                self.duration_label.config(text=self.format_time(self.audio_duration))
                self.progress.config(to=self.audio_duration)
                messagebox.showinfo("Success", "Audio file loaded successfully!")
            else:
                messagebox.showerror("Error", "Failed to load audio file")
        else:
            messagebox.showerror("Error", "Unsupported file format. Please use .wav, .mp3, or .ogg files")
    def open_file(self):
        filetypes = [
            ("Audio files", "*.wav;*.mp3;*.ogg"),
            ("WAV files", "*.wav"),
            ("MP3 files", "*.mp3"),
            ("OGG files", "*.ogg"),
            ("All files", "*.*")
        ]
        
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            if self.equalizer.player.load_file(filename):
                self.file_label.config(text=os.path.basename(filename))
                
                # Set duration label
                self.audio_duration = self.equalizer.player.audio_file.duration
                self.duration_label.config(text=self.format_time(self.audio_duration))
                self.progress.config(to=self.audio_duration)  # set progress bar range

    def save_file(self):
        if self.equalizer.player.audio_file.audio_data is None:
            tk.messagebox.showerror("Error", "No audio file loaded")
            return
            
        original_ext = os.path.splitext(self.equalizer.player.audio_file.filename)[1]
        filetypes = [("WAV files", "*.wav")]
        default_name = f"processed_audio{original_ext}"
        
        filename = filedialog.asksaveasfilename(
            defaultextension=".wav",
            filetypes=filetypes,
            initialfile=default_name
        )
        
        if filename:
            if self.equalizer.save_processed_audio(filename):
                tk.messagebox.showinfo("Success", "Processed audio saved successfully!")
            else:
                tk.messagebox.showerror("Error", "Failed to save processed audio")
                
    def play_audio(self):
        if self.equalizer.player.audio_file.audio_data is not None:
            self.equalizer.player.play(self.equalizer.audio_callback)
            self.update_progress_bar()
            
    def stop_audio(self):
        self.equalizer.player.stop()
        self.equalizer.player.audio_file.current_position = 0
        self.progress.set(0)
        self.current_time_label.config(text="00:00")

    def seek_audio(self, value):
        """Seek to a specific position in the audio."""
        if self.equalizer.player.audio_file.audio_data is not None:
            position = float(value)
            self.equalizer.player.audio_file.seek(position)
            self.current_time_label.config(text=self.format_time(position))
        
    def update_visualization(self):
        while self.running:
            try:
                audio_data = self.equalizer.audio_queue.get_nowait()
                self.visualizer.update(audio_data)
            except queue.Empty:
                time.sleep(0.01)
            except Exception as e:
                print(f"Visualization error: {e}")

    def update_gain(self, band_idx, value):
        self.equalizer.set_gain(band_idx, float(value))
        
    def format_time(self, seconds):
        minutes = int(seconds // 60)
        seconds = int(seconds % 60)
        return f"{minutes:02}:{seconds:02}"

    def update_progress_bar(self):
        """Update the progress bar based on the playback position."""
        def update():
            while self.running:
                # Check if audio is playing and update progress if it is
                if self.equalizer.player.audio_file.is_playing:
                    # Retrieve and set the current position in seconds
                    self.current_time = (
                        self.equalizer.player.audio_file.current_position 
                        / self.equalizer.player.sample_rate
                    )
                    self.progress.set(self.current_time)
                    self.current_time_label.config(text=self.format_time(self.current_time))
                time.sleep(0.5)  # Update every 500ms

        # Run the update in a separate thread if not already running
        if not hasattr(self, "_progress_thread") or not self._progress_thread.is_alive():
            self._progress_thread = threading.Thread(target=update)
            self._progress_thread.daemon = True
            self._progress_thread.start()

    def format_time(self, seconds):
        """Format seconds to MM:SS."""
        minutes = int(seconds // 60)
        seconds = int(seconds % 60)
        return f"{minutes:02}:{seconds:02}"
    
    def run(self):
        try:
            self.root.mainloop()
        finally:
            self.running = False
            self.equalizer.player.stop()

if __name__ == "__main__":
    eq = ThreeBandEqualizer()
    gui = ThreeBandEqualizerGUI(eq)
    app = RealTimeAudioApp()
    gui.run()
    app.run()